In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [3]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [4]:
query = \
"""
select * from icu_features;
"""
res = pd.read_sql_query(query, con)
res.head()

,icustay_id,hadm_id,subject_id,age,gender,height,weight,bmi,ethnicity,insurance,...,ea_tv_pulm_htn,ea_tv_tr,ea_lv_cavity,ea_lv_diastolic,ea_lv_systolic,ea_lv_wall,ea_rv_cavity,ea_rv_diastolic_fluid,ea_rv_systolic,ea_rv_wall
0,287547,189332,109,9146 days 19:49:27,F,152.4,19.368394,8.339186,BLACK/AFRICAN AMERICAN,Medicaid,...,2.0,0.0,0.0,NaN,0.0,3.0,0.0,NaN,0.0,1.0
1,254245,192123,111,24368 days 06:56:33,F,NaN,57.560001,NaN,WHITE,Medicare,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,0.0,NaN
2,224803,130684,225,11035 days 00:58:13,M,NaN,69.800003,NaN,WHITE,Medicaid,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,215044,116935,249,27285 days 20:06:02,F,NaN,72.000000,NaN,WHITE,Medicare,...,1.0,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,257572,111199,281,21906 days 04:45:22,F,NaN,83.650002,NaN,BLACK/AFRICAN AMERICAN,Government,...,2.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN


Check that icustay_id is unique. 

In [5]:
len(res['icustay_id'].unique()) == res.shape[0]

True

+ had sepsis

In [6]:
res['filter_angus_sepsis'].value_counts()

0    44112
1    17420
Name: filter_angus_sepsis, dtype: int64

+ on vasopressors

In [7]:
res['filter_vaso'].value_counts()

False    47716
True     13816
Name: filter_vaso, dtype: int64

+ first care unit or last careunit was micu

In [12]:
res['ic_first_careunit'].value_counts()

MICU     21088
CSRU      9312
SICU      8891
NICU      8100
CCU       7726
TSICU     6415
Name: ic_first_careunit, dtype: int64

In [13]:
res['ic_last_careunit'].value_counts()

MICU     21310
CSRU      9392
SICU      9213
NICU      8100
CCU       7406
TSICU     6111
Name: ic_last_careunit, dtype: int64

+ first or last care unit = MICU, SICU, NICU

In [18]:
pd.Series((res['ic_first_careunit'].apply(lambda u: u in ['MICU', 'SICU', 'NICU']) | 
 res['ic_last_careunit'].apply(lambda u: u in ['MICU', 'SICU', 'NICU']))).value_counts()

True     39392
False    22140
dtype: int64

+ service type

In [23]:
res.loc[res.st_micu == 1].shape

(18855, 113)

In [24]:
res.loc[res.st_nsicu == 1].shape

(4626, 113)

In [25]:
res.loc[res.st_sicu == 1].shape

(9755, 113)

In [28]:
res.loc[(res.st_micu == 1) | (res.st_nsicu == 1) | (res.st_sicu == 1)].shape

(26773, 113)

+ service type = {micu, nsicu, sicu} or first care unit = {micu, nsicu, sicu} or last care unit = {micu, nsicu, sicu}

In [51]:
filter_careunit = (res['ic_first_careunit'].apply(lambda u: u in ['MICU', 'SICU', 'NICU']) | 
    res['ic_last_careunit'].apply(lambda u: u in ['MICU', 'SICU', 'NICU']) |
    (res.st_micu == 1) | (res.st_nsicu == 1) | (res.st_sicu == 1))
pd.Series(filter_careunit).value_counts()

True     42884
False    18648
dtype: int64

+ had sepsis
+ (and/or) was on vasopressors

In [8]:
res.loc[res['filter_vaso'] & res['filter_angus_sepsis']].shape[0]

6182

In [9]:
res.loc[res['filter_vaso'] | res['filter_angus_sepsis']].shape[0]

25054

+ had sepsis or was on vasopressors
+ service type = {micu, nsicu, sicu} or first care unit = {micu, nsicu, sicu} or last care unit = {micu, nsicu, sicu}

In [33]:
res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) & filter_careunit].shape[0]

16028

+ had sepsis or was on vasopressors
+ service type = {micu, nsicu, sicu} or first care unit = {micu, nsicu, sicu} or last care unit = {micu, nsicu, sicu}
+ no cardiogenic shock

In [34]:
res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) 
        & filter_careunit
        & ~res['filter_hard_cardiogenic']
].shape[0]

15058

+ had sepsis or was on vasopressors
+ service type = {micu, nsicu, sicu} or first care unit = {micu, nsicu, sicu} or last care unit = {micu, nsicu, sicu}
+ no cardiogenic shock
+ not on chronic dialysis
+ was adult

In [38]:
res_ = res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) 
        & filter_careunit
        & ~res['filter_hard_cardiogenic']
        & ~res['filter_chronic_dialysis']
        & res['filter_adult']
]
res_.shape[0]

13487

Number of unique subject ids

In [40]:
len(res_['subject_id'].unique())

10363

In [52]:
final = res_.sort_values('intime', ascending = False).groupby(
    ['subject_id', 'filter_echo']).head(n = 1).sort_values(
    'filter_echo', ascending = False).groupby('subject_id').head(n = 1)
final.shape

(10363, 113)

Look at fluids

In [60]:
final[[c for c in final.columns if 'fb' in c]].describe()

,fb_day1_input_ml,fb_day1_output_ml,fb_day1_balance_ml,fb_day2_input_ml,fb_day2_output_ml,fb_day2_balance_ml,fb_day3_input_ml,fb_day3_output_ml,fb_day3_balance_ml
count,9736.000000,9736.000000,9736.000000,8245.000000,8245.000000,8245.000000,6386.000000,6386.000000,6386.000000
mean,3275.741093,2331.097668,944.643425,1851.209322,2032.202323,804.947003,1428.578726,2195.416192,194.533038
std,3054.393438,2874.683574,3906.300721,1811.720496,2641.484621,6115.692466,1438.614493,3091.909729,8833.803311
min,1.100000,0.000000,-122515.833344,0.333332,0.000000,-235268.933342,2.733334,0.000000,-334065.792215
25%,1237.163542,1050.750000,-860.250003,540.000005,975.000000,-1787.000000,380.000000,1050.000000,-3039.625000
50%,2467.312500,1810.000000,476.704163,1395.166334,1660.000000,322.498289,1000.415977,1818.000000,-234.000000
75%,4338.988061,2900.000000,2325.372978,2534.333283,2620.000000,3033.000000,2003.711295,2859.000000,3262.425331
max,31198.375000,123200.000000,27652.375000,22113.447687,130775.000000,36905.903404,15360.500000,161612.000000,45132.314719
